# 9차시 실습: 안전한 에이전트 설계 원칙 (Safe Design)

이 실습에서는 AI 에이전트를 구축할 때 반드시 지켜야 할 **최소 권한**, **입력 검증**, **출력 무해화** 원칙을 코드로 구현해봅니다.

### 학습 목표
1. `eval()` 등 위험한 코드 실행 함수의 위험성 이해
2. `Pydantic`을 활용한 도구 입력값 강제 검증 (Validation)
3. `html.escape`를 활용한 출력 무해화 (Sanitization)
4. 에이전트 행위를 추적할 수 있는 구조화된 로깅 (Logging)

In [ ]:
# 필수 라이브러리 설치
!pip install pydantic

## 1. [Bad Case] 취약한 도구 실행
LLM이 생성한 코드를 검증 없이 실행할 때 발생하는 문제를 시뮬레이션합니다.

In [1]:
import os

# [위험] LLM이 문자열로 코드를 생성했다고 가정합니다.
# 해커가 주입한 악성 코드가 포함되어 있을 수 있습니다.
llm_generated_code = "print('중요 파일 삭제 중...'); os.system('echo File Deleted')"

def unsafe_executor(code):
    print("⚠️ 경고: 검증되지 않은 코드를 실행합니다.")
    # eval이나 exec는 문자열을 그대로 코드로 실행하므로 매우 위험합니다.
    try:
        exec(code)
    except Exception as e:
        print(f"Error: {e}")

# 실행 테스트
unsafe_executor(llm_generated_code)

⚠️ 경고: 검증되지 않은 코드를 실행합니다.
중요 파일 삭제 중...


## 2. [Good Case] Pydantic을 이용한 입력 검증
도구(Tool)가 받을 수 있는 인자(Argument)의 타입과 범위를 엄격하게 제한합니다.

In [2]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional

# 1. 도구의 입력 스키마(규칙) 정의
class TransferMoneySchema(BaseModel):
    account_number: str = Field(..., pattern=r"^\d{3}-\d{6}$", description="계좌번호 형식 (000-000000)")
    amount: int = Field(..., gt=0, le=1000000, description="송금액 (0원 초과, 100만원 이하)")
    memo: Optional[str] = Field(None, max_length=10, description="메모 (10자 이내)")

def safe_transfer_tool(arguments: dict):
    try:
        # 2. Pydantic을 사용하여 입력값 검증 수행
        validated_data = TransferMoneySchema(**arguments)
        
        # 검증 통과 시 로직 실행
        print(f"✅ [성공] {validated_data.amount}원을 {validated_data.account_number}로 송금합니다.")
        
    except ValidationError as e:
        # 검증 실패 시 에러 메시지 출력
        print(f"🚫 [차단] 유효하지 않은 입력입니다:\n{e}")

# 테스트 1: 정상 입력
print("--- 정상 입력 테스트 ---")
safe_transfer_tool({"account_number": "123-456789", "amount": 50000, "memo": "점심값"})

# 테스트 2: 악성/오류 입력 (금액 초과, 계좌번호 형식 오류)
print("\n--- 오류 입력 테스트 ---")
safe_transfer_tool({"account_number": "invalid-acc", "amount": 50000000}) # 5천만원(한도초과)

--- 정상 입력 테스트 ---
✅ [성공] 50000원을 123-456789로 송금합니다.

--- 오류 입력 테스트 ---
🚫 [차단] 유효하지 않은 입력입니다:
2 validation errors for TransferMoneySchema
account_number
  String should match pattern '^\d{3}-\d{6}$' [type=string_pattern_mismatch, input_value='invalid-acc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_pattern_mismatch
amount
  Input should be less than or equal to 1000000 [type=less_than_equal, input_value=50000000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal


## 3. [Sanitization] 출력 무해화
에이전트가 생성한 텍스트를 웹 브라우저 등에 표시하기 전, 악성 스크립트를 무력화합니다.

In [3]:
import html

# LLM이 생성한 답변 (XSS 공격 코드 포함)
unsafe_output = "안녕하세요! <script>alert('당신의 쿠키를 훔쳤습니다!');</script>"

def sanitize_output(text):
    # HTML 특수 문자를 이스케이프 처리 (< -> &lt;)
    safe_text = html.escape(text)
    return safe_text

print(f"[원본] {unsafe_output}")
print(f"[무해화] {sanitize_output(unsafe_output)}")

[원본] 안녕하세요! <script>alert('당신의 쿠키를 훔쳤습니다!');</script>
[무해화] 안녕하세요! &lt;script&gt;alert(&#x27;당신의 쿠키를 훔쳤습니다!&#x27;);&lt;/script&gt;


## 4. [Logging] 구조화된 감사 로그(Audit Log)
보안 사고 발생 시 추적할 수 있도록 '누가, 언제, 무엇을' 했는지 기록합니다.

In [4]:
import logging
import json
from datetime import datetime

# 로거 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger("AgentAudit")

def log_agent_action(user_id, tool_name, tool_args, status):
    # 구조화된 JSON 로그 생성
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "user_id": user_id,
        "action": "tool_execution",
        "tool": tool_name,
        "arguments": tool_args,
        "status": status
    }
    # 실제로는 파일이나 DB로 저장하지만, 여기선 콘솔 출력
    logger.info(json.dumps(log_entry, indent=2, ensure_ascii=False))

# 로그 테스트
log_agent_action("user_001", "transfer_money", {"amount": 10000}, "success")

{
  "timestamp": "2026-01-03T20:05:28.813875",
  "user_id": "user_001",
  "action": "tool_execution",
  "tool": "transfer_money",
  "arguments": {
    "amount": 10000
  },
  "status": "success"
}
